In [ ]:
import emission_line_luminosity
from emission_line_luminosity import get_emission_line_luminosity
from emission_line_luminosity import _get_clipped_sed
from emission_line_luminosity import _quad_continuum_model
import dsps
from dsps import load_ssp_templates
from dsps.cosmology import age_at_z, DEFAULT_COSMOLOGY
from dsps import calc_rest_sed_sfh_table_lognormal_mdf
from dsps import calc_rest_sed_sfh_table_met_table
import numpy as np
from matplotlib import pyplot as plt

from jax import jit as jjit
from jax import grad
import jax.numpy as jnp
from jax import vmap 
from tqdm.autonotebook import tqdm
from astropy.constants import L_sun
import astropy.constants as const
import matplotlib.cm as cm

import astropy.units as u
import numpy.ma as ma
import h5py

In [ ]:
# ! curl https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/ssp_data_fsps_v3.2_lgmet_age.h5 > tempdata.h5

In [ ]:
ssp_data = load_ssp_templates(fn='./workdir/tempdata.h5')

print(ssp_data._fields)

print('\nssp_lgmet.shape = {}'.format(ssp_data.ssp_lgmet.shape))
print('ssp_lg_age_gyr.shape = {}'.format(ssp_data.ssp_lg_age_gyr.shape))
print('ssp_wave.shape = {}'.format(ssp_data.ssp_wave.shape))
print('ssp_flux.shape = {}'.format(ssp_data.ssp_flux.shape))

In [ ]:
ssp_halpha_line_luminosity = np.zeros((ssp_data.ssp_lgmet.shape[0], ssp_data.ssp_lg_age_gyr.shape[0]))

In [ ]:
ssp_halpha_line_luminosity.shape

In [ ]:
for met in range(0, ssp_halpha_line_luminosity.shape[0]):
    for age in range(0, ssp_halpha_line_luminosity.shape[1]):
        plt.loglog(ssp_data.ssp_wave, ssp_data.ssp_flux[met][age])
plt.show()

## Calculate H$\alpha$ line luminosty for the SSP grid (metallicity, age) : L$_{H\alpha}$ [Lsun/Msun]

In [ ]:
continuum_fit_lo_lo = 6480
continuum_fit_lo_hi = 6530
continuum_fit_hi_lo = 6600
continuum_fit_hi_hi = 6650

halpha_line_center = 6562.8
halpha_line_lo = halpha_line_center - 2
halpha_line_hi = halpha_line_center + 6

In [ ]:
for met in range(0, ssp_halpha_line_luminosity.shape[0]):
    for age in range(0, ssp_halpha_line_luminosity.shape[1]):
        losses, theta, sed_dict = get_emission_line_luminosity(ssp_data.ssp_wave,
                                                               ssp_data.ssp_flux[met][age],
                                                               continuum_fit_lo_lo,
                                                               continuum_fit_lo_hi,
                                                               continuum_fit_hi_lo,
                                                               continuum_fit_hi_hi,
                                                               halpha_line_lo,
                                                               halpha_line_hi
                                                              )
        ssp_halpha_line_luminosity[met][age] = sed_dict['line_integrated_luminosity']

In [ ]:
# with h5py.File("./workdir/tempdata.h5", "w") as f:
#     if "ssp_halpha_line_luminosity" not in f:
#         f.create_dataset("ssp_halpha_line_luminosity", data=ssp_halpha_line_luminosity)

In [ ]:
with h5py.File("./workdir/halpha.h5", "r") as f:
    print("Keys in file:", list(f.keys()))   # list top-level groups/datasets
    dset = f['ssp_halpha_line_luminosity']
    ssp_halpha_line_luminosity = dset[...]    

In [ ]:
# np.allclose(ssp_halpha_line_luminosity_read, ssp_halpha_line_luminosity)

In [ ]:
# ssp_halpha_line_luminosity.shape, ssp_halpha_line_luminosity.dtype

In [ ]:
colors=cm.coolwarm(np.linspace(1, 0, ssp_halpha_line_luminosity.shape[0])) # red first

In [ ]:
for met in range(0, ssp_halpha_line_luminosity.shape[0]):
    metallicity = str(np.round(ssp_data.ssp_lgmet[met],1))
    x = (10**ssp_data.ssp_lg_age_gyr)*1e3
    y = ssp_halpha_line_luminosity[met] #* L_sun.cgs.value * 1e9
    label = 'log$_{10}$ (Z/Z$_{\u2609}$): '+metallicity
    
    plt.plot(x,y, label=label, c=colors[met])
plt.ylabel('L$_{H\u03b1}$ [L$_{\u2609}$ / M$_{\u2609}$]')
plt.xscale('log')
plt.xlabel('Age [Myr]')
#plt.xlim(8e-2,1e5)
plt.legend()
plt.savefig('./workdir/figures/Age_v_Lhalpha.pdf')

plt.show()

In [ ]:
pwd